# Pre-Processing of MIF180 $\lambda$ Spacing Data

The cells below are examples of the slow analyses (run in TMUX sessions using ipython) which were performed to generate data for the final analyses.

In [21]:
import a3fe as ee 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams, rcParamsDefault
import seaborn as sns
import pickle as pkl
from typing import List, Tuple, Dict, Callable, Union
from scipy import stats
import numpy as np
from EnsEquil.read._process_somd_files import read_overlap_mat
import matplotlib.colors as colors
import BioSimSpace as BSS
rcParams.update(rcParamsDefault)
plt.style.use("seaborn-v0_8-colorblind")
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')
%matplotlib inline

## Run in TMUX session

In [ ]:
import a3fe as a3
import numpy as np
import pickle as pkl

base_dirs = ['mif_180_anti_opt_lam_05_non_adapt',
 'mif_180_anti_opt_lam_1_non_adapt',
 'mif_180_anti_opt_lam_2_non_adapt',
 'mif_180_anti_opt_lam_4_non_adapt']

stage_types = {"restrain": a3.StageType.RESTRAIN, "discharge": a3.StageType.DISCHARGE, "vanish": a3.StageType.VANISH}

conv_dict = {}
for leg in ["bound", "free"]:
    conv_dict[leg] = {}
    paths = [p + "/" + leg for p in base_dirs]
    stages = ["discharge", "vanish"]
    stages += ["restrain"] if leg == "bound" else []
    for stage in stages:
        stage_paths = [p + "/" + stage for p in paths]
        stage_iterator = a3.run._utils.SimulationRunnerIterator(stage_paths, a3.Stage, stage_type = stage_types[stage])
        res = a3.analyse.get_comparitive_convergence_data(stage_iterator, mode="block")
        conv_dict[leg][stage] = res

    with open("final_analysis/overall_conv_dict.pkl", "wb") as f:
        pkl.dump(conv_dict, f)


In [ ]:
import a3fe as a3
import numpy as np
import pickle as pkl

base_dirs = ['mif_180_anti_opt_lam_05_non_adapt',
 'mif_180_anti_opt_lam_1_non_adapt',
 'mif_180_anti_opt_lam_2_non_adapt',
 'mif_180_anti_opt_lam_4_non_adapt',
 '../non_adaptive_final/mif_180_anti_5000ps',
 '../non_adaptive_final/mif_180_anti_30000ps',]

stage_types = {"restrain": a3.StageType.RESTRAIN, "discharge": a3.StageType.DISCHARGE, "vanish": a3.StageType.VANISH}

conv_dict = {}
for leg in ["bound", "free"]:
    conv_dict[leg] = {}
    paths = [p + "/" + leg for p in base_dirs]
    stages = ["discharge", "vanish"]
    stages += ["restrain"] if leg == "bound" else []
    for stage in stages:
        stage_paths = [p + "/" + stage for p in paths]
        stage_iterator = a3.run._utils.SimulationRunnerIterator(stage_paths, a3.Stage, stage_type = stage_types[stage])
        res = a3.analyse.get_comparitive_convergence_data(stage_iterator, mode="block")
        conv_dict[leg][stage] = res

    with open("final_analysis/conv_dict_incl_default_spacing.pkl", "wb") as f:
        pkl.dump(conv_dict, f)


In [ ]:
import a3fe as a3
import numpy as np
import pickle as pkl

base_dirs = ["mif_180_anti_opt_lam_05_non_adapt",
             "mif_180_anti_opt_lam_1_non_adapt",
             "mif_180_anti_opt_lam_2_non_adapt",
             "mif_180_anti_opt_lam_4_non_adapt"]

free_vanish_dirs = [d + "/free/vanish" for d in base_dirs]
free_vanish_iterators = a3.run._utils.SimulationRunnerIterator(free_vanish_dirs, a3.Stage, stage_type = a3.StageType.VANISH)
free_vanish_conv_002 = a3.analyse.get_comparitive_convergence_data(free_vanish_iterators, mode="block", frac=0.02)

with open("final_analysis/mif_free_vanish_conv_002_with_4.pkl", "wb") as f:
    pkl.dump(free_vanish_conv_002, f)

In [ ]:
base_dirs = {"non_adaptive_6": "../non_adaptive_final/mif_180_anti_5000ps",
             "non_adaptive_30": "../non_adaptive_final/mif_180_anti_30000ps",
             "0.5": "mif_180_anti_opt_lam_05_non_adapt",
             "1.0": "mif_180_anti_opt_lam_1_non_adapt",
             "2.0": "mif_180_anti_opt_lam_2_non_adapt",
             "4.0": "mif_180_anti_opt_lam_4_non_adapt"
            }


overlap_mats = {}
for spacing, base_dir in base_dirs.items():
    overlap_mats[spacing] = {}
    for leg in ["bound", "free"]:
        overlap_mats[spacing][leg] = {}
        path_leg = base_dir + "/" + leg
        stages = ["discharge", "vanish"]
        stages += ["restrain"] if leg == "bound" else []
        for stage in stages:
            path_stage = path_leg + "/" + stage
            overlap_mats[spacing][leg][stage] = []
            for run in range(1, 6):
                if spacing == "non_adaptive_30":
                    mbar_addition = f"/output/freenrg-MBAR-run_0{run}_100_percent.dat"
                else:
                    mbar_addition = f"/output/freenrg-MBAR-run_0{run}_100_end_0_start.dat"
                mbar_path = path_stage + mbar_addition
                overlap_mat = read_overlap_mat(mbar_path)
                overlap_mats[spacing][leg][stage].append(overlap_mat)
            # Get the mean overlap matrix
            overlap_mats[spacing][leg][stage] = np.mean(overlap_mats[spacing][leg][stage], axis=0)

with open("final_analysis/overlap_mats.pkl", "wb") as f:
    pkl.dump(overlap_mats, f)

In [ ]:
# Get the lambda values for each of the simulations

base_dirs = {"non_adaptive_6": "../non_adaptive_final/mif_180_anti_5000ps",
             "non_adaptive_30": "../non_adaptive_final/mif_180_anti_30000ps",
             "0.5": "mif_180_anti_opt_lam_05_non_adapt",
             "1.0": "mif_180_anti_opt_lam_1_non_adapt",
             "2.0": "mif_180_anti_opt_lam_2_non_adapt",
             "4.0": "mif_180_anti_opt_lam_4_non_adapt"
             }

lam_vals = {}
for name, base_dir in base_dirs.items():
    lam_vals[name] = {}
    calc = a3.Calculation(base_dir=base_dir)
    for leg in calc.legs:
        lam_vals[name][leg.leg_type.name.lower()] = {}
        for stage in leg.stages:
            lam_vals[name][leg.leg_type.name.lower()][stage.stage_type.name.lower()] = stage.lam_vals

    calc._close_logging_handlers()
    del(calc)

with open("final_analysis/lam_vals.pkl", "wb") as f:
    pkl.dump(lam_vals, f)